In [7]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [8]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [12]:
include("utils.jl")

get_eof_of_datachunk (generic function with 1 method)

In [10]:
base_path = "/home/denis/workspace/data/ivt_fields_v1"

"/home/denis/workspace/data/ivt_fields_v1"

In [13]:
winter_months = [12, 1, 2, 3]

tl_data = filter_by_date(build_timeline_data(base_path, 10, "ssp126", "ssp585"; file_range_selection = 4:5)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2075-01-01T06:00:00"), DateTime("2075-01-01T12:00:00"), DateTime("2075-01-01T18:00:00"), DateTime("2075-01-02T00:00:00"), DateTime("2075-01-02T06:00:00"), DateTime("2075-01-02T12:00:00"), DateTime("2075-01-02T18:00:00"), DateTime("2075-01-03T00:00:00"), DateTime("2075-01-03T06:00:00"), DateTime("2075-01-03T12:00:00")  …  DateTime("2100-12-29T18:00:

In [22]:

# create geographical axes
function local_geoaxis_creation!(
    figure::Makie.Figure,
    lonlims::Tuple{<:Number, <:Number}, 
    latlims::Tuple{<:Number, <:Number}; 
    figure_row::Int = 1,
    figure_col::Int = 1,
    title = "",
    source_projection = "+proj=longlat +datum=WGS84",
    dest_projection = "+proj=merc",
    )

    geoaxis = GeoMakie.GeoAxis(
        figure[figure_row, figure_col]; 
        dest = dest_projection,
        source = source_projection,
        limits=(lonlims, latlims),
        title = title
    )

    return geoaxis
end


function build_figure(
    data,
    lon_bounds::Tuple{<:Number, <:Number},
    lat_bounds::Tuple{<:Number, <:Number};
    title::String = "",
    colormap = :viridis,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    

    ga = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; title=title)
    surface!(ga, lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], data; shading = shading, colormap = colormap)
    lines!(ga, GeoMakie.coastlines(); color = :white, transformation = (; translation = (0, 0, 1000)))
    return fig
end

build_figure (generic function with 1 method)

In [26]:
build_figure(data_filtered[:, :, 1], (-90, 40), (20, 80); title = "Test IVT 1", colormap = :managua100)

In [20]:

function animate_timeline(
    data::TimelineData,
    filename::String;
    framerate::Int = 30,
    colormap = :viridis,
    coastline_color = :white,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    time_index = Observable(1)

    axis = Dict()

    lon_bounds = extrema(data.lons)
    lat_bounds = extrema(data.lats)

    all_extrema = extrema.([d.data for d in data.datasets])

    min_val, max_val = reduce((a, b) -> (min(a[1], b[1]), max(a[2], b[2])), all_extrema)
    Colorbar(fig[2, 1:2], limits = (min_val, max_val), colormap = colormap, vertical = false,  label = "IVT (norm) in kg s^-1 m^-1")

    for (i, dataset) in enumerate(data.datasets)
        axis[dataset.name] = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; title=@lift("$(dataset.name) $(Dates.format(data.time[$time_index], "mm/yyyy"))"), figure_col=i)
    end
    
    for dataset in data.datasets

        array_slice = @lift(dataset.data[:, :, $time_index])
        
        surface!(axis[dataset.name], lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], array_slice; shading = shading, colormap = colormap, colorrange = (min_val, max_val))
        lines!(axis[dataset.name], GeoMakie.coastlines(); color = coastline_color, transformation = (; translation = (0, 0, 1000)))
    end

    timestamps = eachindex(data.time)
    record(fig, filename, timestamps; framerate = framerate) do t
        time_index[] = t
    end

    
    return fig
end

animate_timeline (generic function with 1 method)

In [23]:
# animate_timeline(tline_data, "scenario_comparison_2.mp4"; colormap = :managua100, resolution = (1000, 1000))
animate_timeline(tl_data, "scenario_comparison_end.mp4"; colormap = :managua100)